# Chapter 14: State Space Techniques in Controller Design

## Example 14.1: Pole_placement_controller_for_inverted_pendulum.sce

In [ ]:
// Pole placement controller for inverted pendulum, discussed in Example 14.1 on page 490. 2.1 should be executed before starting this code
// 14.1

exec('pol2cart.sci',-1);

C = eye(4,4);
D = zeros(4,1);
Ts = 0.01;
G = syslin('c',A,B,C,D);
H = dscr(G,Ts);
[a,b,c,d] = H(2:5);
rise = 5; epsilon = 0.1;
N = rise/Ts;
omega = %pi/2/N;
r = epsilon^(omega/%pi); 
r1 = r; r2 = 0.9*r;
[x1,y1] = pol2cart(omega,r1);
[x2,y2] = pol2cart(omega,r2);
p1 = x1+%i*y1;
p2 = x1-%i*y1;
p3 = x2+%i*y2;
p4 = x2-%i*y2;
P = [p1;p2;p3;p4];
K = ppol(a,b,P)

## Example 14.2: Compensator_calculation.sce

In [ ]:
// Compensator calculation for Example 14.6 on page 507.
// 14.2

exec('polyno.sci',-1);
exec('polmul.sci',-1);
exec('polsize.sci',-1);

A = [1 2; 0 3]; c = [1 0];
p = roots(polyno([1 -0.5 0.5],'z'));
b = [0; 1];
K = ppol(A,b,p);

p1=0.1+0.1*%i; p2=0.1-0.1*%i;
phi = real(convol([1 -p1],[1 -p2]));
Obs = [c; c*A];
alphae = A^2-0.2*A+0.02*eye(2,2);
Lp = alphae*inv(Obs)*[0; 1];
Lp = ppol([1 0;2 3], ...
[1; 0],[0.1+0.1*%i 0.1-0.1*%i]);
Lp = Lp';

C = [1 0 0.5 2;0 1 -4.71 2.8]; 
dC = 1;

[HD,dHD] = polmul(K,0,C,dC);
[HD,dHD] = polmul(HD,dHD,Lp,0);

## Example 14.3: Kalman_filter_example_of_estimating_a_constant.sce

In [ ]:
// Kalman filter example of estimating a constant, discussed in Example 14.7.
// 14.3

exec('kal_ex.sci',-1);

x = 5; xhat = 2; P = 1; xvec = x; 
xhat_vec = xhat; Pvec = P; yvec = x;
for i = 1:200,
    xline = xhat; M = P;
    [xhat,P,y] = kal_ex(x,xline,M);
    xvec = [xvec;x]; 
    xhat_vec = [xhat_vec;xhat]; 
    Pvec = [Pvec;P]; yvec = [yvec;y];
end
n = 1:201;
plot(Pvec);
xtitle('','n');
halt();
clf();
plot(n,xhat_vec',n,yvec',n,xvec');
xtitle('','n');

## Example 14.4: Kalman_filter_example_of_estimating_a_constant.sci

In [ ]:
// Kalman filter example of estimating a constant
// 14.4

function [xhat,P,y] = kal_ex(x,xline,M)
y = x + rand();
Q = 0; R = 1;
xhat_ = xline;
P_ = M + Q;
K = P_/(P_+R);
P = (1-K)*P_;
xhat = xhat_ + K*(y-xhat_);
endfunction;